In [8]:
import os
import json
import numpy as np

data_dir = 'G:/signData'

train_dir = os.path.join(data_dir,'train')
val_dir = os.path.join(data_dir,'valid')

output_dir = os.path.join(data_dir,'nptxt')
weight_dir = os.path.join(data_dir,'weights')
val_output_dir = os.path.join(data_dir,"nptxt_val")

train_landmark_dir = os.path.join(train_dir,'label','landmark')
train_morpheme_dir = os.path.join(train_dir,'label','morpheme')

val_landmark_dir = os.path.join(val_dir,'landmark')
val_morpheme_dir = os.path.join(val_dir,'morpheme')

In [9]:
def weight_calc(F, a, b, x):
    fps=30
    a_frame = a * fps  # a를 프레임 단위로 변환
    b_frame = b * fps  # b를 프레임 단위로 변환
    x_frame = x * fps  # x(영상의 길이)를 프레임 단위로 변환
    if F < a_frame:
        return 0
    elif a_frame <= F < a_frame + 2 * fps:
        return (F - a_frame) / (2 * fps)
    elif a_frame + 2 * fps <= F < b_frame:
        return 1
    elif b_frame <= F <= x_frame:
        return 1 - ((F - b_frame) / (x_frame - b_frame)) * 0.5
    else:
        return 0.5
	

In [1]:
import tensorflow as tf
print(tf.__version__)

AttributeError: module 'tensorflow' has no attribute '__version__'

In [9]:
|print(weight_calc(113,1.743,3.103,4.867))

0.811885865457294


In [4]:
#train weight data
start_person="01"
start_word="0001"
#사람별 데이터(01~16)
for person in os.listdir(train_landmark_dir):
    #중간부터 다시 데이터 변환시 체크포인트
    if int(person) < int(start_person):
        continue
        
    #변환 데이터 저장시 사람 폴더 생성(01~16)
    person_output_path = os.path.join(weight_dir, str(int(person)))
    os.makedirs(person_output_path, exist_ok=True)
    
    #각 단어별 좌표값 및 단어 뜻 순회. 
    for word_coords, word_morpheme in zip(os.listdir(os.path.join(train_landmark_dir, person)), os.listdir(os.path.join(train_morpheme_dir, person))):
        #중간부터 다시 데이터 변환시 체크 포인트
        if int(word_coords[11:15]) < int(start_word):
            continue
        #정면카메라 외 다른 방향 카메라도 3d좌표값은 동일하여 한개의 방향에서만 데이터 추출
        #단어 별 뜻, 단어의 할당 넘버, 영상 길이와 영상 내 수어 구간 추출
        if "F" in word_morpheme:
            file_path = os.path.join(train_morpheme_dir, person, word_morpheme)
            morpheme_file_path = os.path.join(train_morpheme_dir, person, word_morpheme)
            with open(morpheme_file_path, 'r', encoding="UTF8") as morpheme_file:
                data = json.load(morpheme_file)
                try:
                    num = data['metaData']['name'][11:15]
                    start_sign = data["data"][0]["start"]
                    end_sign = data["data"][0]["end"]
                    duration = data["metaData"]["duration"]
                #결측치 오류 검출   
                except IndexError as e:
                    name = False
                    print(f"Error reading {morpheme_file_path}: {e}")
                    continue
        
        #각 단어 별 좌표 순회시, 단어 뜻이 결측치인 데이터는 제외
        if "F" in word_coords and name is not False:
            wordWeight = []
            #단어 영상의 각 프레임별 좌표값 추출
            for frame in os.listdir(os.path.join(train_landmark_dir, person, word_coords)):
                frame_num = int(frame[25:37])
                # 가중치 계산만 수행
                wordWeight.append(weight_calc(frame_num, start_sign, end_sign, duration))
            
            # 결과를 저장
            time = [start_sign, end_sign, duration]
            word_output_path = os.path.join(person_output_path, f'{num}.npz')
            np.savez(word_output_path, weight=wordWeight, time=time)
            print(f"Saved weights for {word_output_path}")
        else:
            continue

FileNotFoundError: [Errno 2] No such file or directory: 'G:/signData/train/label/landmark'

In [4]:
#train data
start_person="01"
start_word="0001"
#사람별 데이터(01~16)
for person in os.listdir(train_landmark_dir):
    #중간부터 다시 데이터 변환시 체크포인트
    if int(person) < int(start_person):
        continue
        
    #변환 데이터 저장시 사람 폴더 생성(01~16)
    person_output_path = os.path.join(output_dir, str(int(person)))
    os.makedirs(person_output_path, exist_ok=True)
    
    #각 단어별 좌표값 및 단어 뜻 순회. 
    for word_coords,word_morpheme in zip(os.listdir(os.path.join(train_landmark_dir, person)),os.listdir(os.path.join(train_morpheme_dir, person))):
        #중간부터 다시 데이터 변환시 체크 포인트
        if int(word_coords[11:15]) <int(start_word):
            continue
        #정면카메라 외 다른 방향 카메라도 3d좌표값은 동일하여 한개의 방향에서만 데이터 추출
        #단어 별 뜻, 단어의 할당 넘버,영상 길이와 영상 내 수어 구간 추출
        if "F" in word_morpheme:
            file_path = os.path.join(train_morpheme_dir, person, word_morpheme)
            morpheme_file_path = os.path.join(train_morpheme_dir, person, word_morpheme)
            with open(morpheme_file_path, 'r', encoding="UTF8") as morpheme_file:
                data = json.load(morpheme_file)
                try:
                    name = data['data'][0]['attributes'][0]['name']
                    num = data['metaData']['name'][11:15]
                    start_sign = data["data"][0]["start"]
                    end_sign = data["data"][0]["end"]
                    duration = data["metaData"]["duration"]
                #결측치 오류 검출   
                except IndexError as e:
                    name=False
                    print(f"Error reading {morpheme_file_path}: {e}")
                    continue
        #각 단어 별 좌표 순회시, 단어 뜻이 결측치인 데이터는 제외
        if "F" in word_coords and name is not False:
            wordCoordL = np.empty((0, 4, 5, 3))  
            wordCoordR = np.empty((0, 4, 5, 3))
            wordCoordP = np.empty((0, 3, 10))
            wordWeight = []
            #단어 영상의 각 프레임별 좌표값 추출            
            for frame in os.listdir(os.path.join(train_landmark_dir, person, word_coords)):
                
                file_path = os.path.join(train_landmark_dir, person, word_coords, frame)
                frame_num=int(frame[25:37])
                with open(file_path, 'r') as json_file:
                    data = json.load(json_file)
                    lh_points = data['people']['hand_left_keypoints_3d']
                    rh_points = data['people']['hand_right_keypoints_3d']
                    p_points = data['people']['pose_keypoints_3d']
                    #포즈 좌표값 추출
                    #1920x1080영상을 메디어파이프의 특성상 영상의 중심을 기준으로 1080x1080으로 잘라서 기존 죄표를 픽셀 값으로 변환 후 
                    # 다시 미디어 파이프와 같은 스케일인 0~1사이 값으로 정규화
                    #z좌표는 미디어파이프의 기준이 되는 랜드마크의 좌표 활용하여 계산
                    
                    preFrameCoordP = np.array([[(960 * p_points[i] + 960 - 420) / (1500 - 420), 
                                                (1080 * p_points[i + 1] + 540) / 1080,
                                                (p_points[32 + 2] - p_points[i + 2]) / 10]
                                               for i in range(0, len(p_points), 4)], dtype=np.float32)

                    preFrameCoordL = np.array([[(960 * lh_points[i] + 960 - 420) / (1500 - 420),
                                                (1080 * lh_points[i + 1] + 540) / 1080,
                                                (lh_points[2] - lh_points[i + 2]) / 10]
                                               for i in range(4, len(lh_points), 4)], dtype=np.float32)

                    preFrameCoordR = np.array([[(960 * rh_points[i] + 960 - 420) / (1500 - 420),
                                                (1080 * rh_points[i + 1] + 540) / 1080,
                                                (rh_points[2] - rh_points[i + 2]) / 10]
                                               for i in range(4, len(rh_points), 4)], dtype=np.float32)
                    
                    #포즈 좌표에서 하반신 랜드마크 제외
                    preFrameCoordP = preFrameCoordP[[i for i in range(19) if (0 <= i <= 7) or (17 <= i <= 18)]]
                    
                    #랜드마크 값들을 모델 트레이닝에 적합한 모양으로 재배열.
                    frameCoordL=preFrameCoordL.reshape(5,4,3).transpose(1,0,2)[::-1]
                    frameCoordR=preFrameCoordR.reshape(5,4,3).transpose(1,0,2)[::-1]
                    frameCoordP=preFrameCoordP.T
                    #한 단어의 프레임 별 좌표를 하나의 넘파이 배열로 저장
                    wordCoordL = np.append(wordCoordL, [frameCoordL], axis=0)
                    wordCoordR = np.append(wordCoordR, [frameCoordR], axis=0)
                    wordCoordP = np.append(wordCoordP, [frameCoordP], axis=0)
                    wordWeight.append(weight_calc(frame_num,start_sign,end_sign,duration))
                    
            label=[name,start_sign,end_sign,duration]
            word_output_path = os.path.join(person_output_path, f'{num}.npz')
            np.savez(word_output_path, wordCoordL=wordCoordL, wordCoordR=wordCoordR, wordCoordP=wordCoordP, label=label, weight=wordWeight)
            print(f"Saved {word_output_path}")
            count+=1
        else:
            continue


FileNotFoundError: [Errno 2] No such file or directory: 'G:/signData/train/label/landmark'

In [6]:
#val data
start_person="01"
start_word="0001"
#사람별 데이터(01~16)
for person in os.listdir(val_landmark_dir):  # train_landmark_dir -> val_landmark_dir
    #중간부터 다시 데이터 변환시 체크포인트
    if int(person) < int(start_person):
        continue
        
    #변환 데이터 저장시 사람 폴더 생성(01~16)
    person_output_path = os.path.join(val_output_dir, str(int(person)))
    os.makedirs(person_output_path, exist_ok=True)
    
    #각 단어별 좌표값 및 단어 뜻 순회. 
    for word_coords,word_morpheme in zip(os.listdir(os.path.join(val_landmark_dir, person)),os.listdir(os.path.join(val_morpheme_dir, person))):  # train_landmark_dir -> val_landmark_dir, train_morpheme_dir -> val_morpheme_dir
        #중간부터 다시 데이터 변환시 체크 포인트
        if int(word_coords[11:15]) < int(start_word):
            continue
        #정면카메라 외 다른 방향 카메라도 3d좌표값은 동일하여 한개의 방향에서만 데이터 추출
        #단어 별 뜻, 단어의 할당 넘버,영상 길이와 영상 내 수어 구간 추출
        if "F" in word_morpheme:
            file_path = os.path.join(val_morpheme_dir, person, word_morpheme)  # train_morpheme_dir -> val_morpheme_dir
            morpheme_file_path = os.path.join(val_morpheme_dir, person, word_morpheme)  # train_morpheme_dir -> val_morpheme_dir
            with open(morpheme_file_path, 'r', encoding="UTF8") as morpheme_file:
                data = json.load(morpheme_file)
                try:
                    name = data['data'][0]['attributes'][0]['name'].replace('\n', '')
                    num = data['metaData']['name'][11:15]
                #결측치 오류 검출   
                except IndexError as e:
                    name = False
                    print(f"Error reading {morpheme_file_path}: {e}")
                    continue
        #각 단어 별 좌표 순회시, 단어 뜻이 결측치인 데이터는 제외
        if "F" in word_coords and name is not False:
            wordCoordL = np.empty((0, 4, 5, 3))  
            wordCoordR = np.empty((0, 4, 5, 3))
            wordCoordP = np.empty((0, 3, 10))
            wordWeight = []
            #단어 영상의 각 프레임별 좌표값 추출            
            for frame in os.listdir(os.path.join(val_landmark_dir, person, word_coords)):  # train_landmark_dir -> val_landmark_dir
                file_path = os.path.join(val_landmark_dir, person, word_coords, frame)  # train_landmark_dir -> val_landmark_dir
                frame_num = int(frame[25:37])
                with open(file_path, 'r') as json_file:
                    data = json.load(json_file)
                    lh_points = data['people']['hand_left_keypoints_3d']
                    rh_points = data['people']['hand_right_keypoints_3d']
                    p_points = data['people']['pose_keypoints_3d']
                    #포즈 좌표값 추출 및 정규화 코드 동일
                    preFrameCoordP = np.array([[(960 * p_points[i] + 960 - 420) / (1500 - 420), 
                                                (1080 * p_points[i + 1] + 540) / 1080,
                                                (p_points[32 + 2] - p_points[i + 2]) / 10]
                                               for i in range(0, len(p_points), 4)], dtype=np.float32)

                    preFrameCoordL = np.array([[(960 * lh_points[i] + 960 - 420) / (1500 - 420),
                                                (1080 * lh_points[i + 1] + 540) / 1080,
                                                (lh_points[2] - lh_points[i + 2]) / 10]
                                               for i in range(4, len(lh_points), 4)], dtype=np.float32)

                    preFrameCoordR = np.array([[(960 * rh_points[i] + 960 - 420) / (1500 - 420),
                                                (1080 * rh_points[i + 1] + 540) / 1080,
                                                (rh_points[2] - rh_points[i + 2]) / 10]
                                               for i in range(4, len(rh_points), 4)], dtype=np.float32)
                    
                    #포즈 좌표에서 하반신 랜드마크 제외
                    preFrameCoordP = preFrameCoordP[[i for i in range(19) if (0 <= i <= 7) or (17 <= i <= 18)]]
                    
                    #랜드마크 값들을 모델 트레이닝에 적합한 모양으로 재배열.
                    frameCoordL = preFrameCoordL.reshape(5, 4, 3).transpose(1, 0, 2)[::-1]
                    frameCoordR = preFrameCoordR.reshape(5, 4, 3).transpose(1, 0, 2)[::-1]
                    frameCoordP = preFrameCoordP.T
                    #한 단어의 프레임 별 좌표를 하나의 넘파이 배열로 저장
                    wordCoordL = np.append(wordCoordL, [frameCoordL], axis=0)
                    wordCoordR = np.append(wordCoordR, [frameCoordR], axis=0)
                    wordCoordP = np.append(wordCoordP, [frameCoordP], axis=0)

                    
            label = [int(person),num,name]
            word_output_path = os.path.join(person_output_path, f'{num}.npz')
            np.savez(word_output_path, wordCoordL=wordCoordL, wordCoordR=wordCoordR, wordCoordP=wordCoordP, label=label)
            print(f"Saved {word_output_path}")
        else:
            continue

Saved G:/signData\nptxt_val\17\0001.npz
Saved G:/signData\nptxt_val\17\0002.npz
Saved G:/signData\nptxt_val\17\0003.npz
Saved G:/signData\nptxt_val\17\0004.npz
Saved G:/signData\nptxt_val\17\0005.npz
Saved G:/signData\nptxt_val\17\0006.npz
Saved G:/signData\nptxt_val\17\0007.npz
Saved G:/signData\nptxt_val\17\0008.npz
Saved G:/signData\nptxt_val\17\0009.npz
Saved G:/signData\nptxt_val\17\0010.npz
Saved G:/signData\nptxt_val\17\0011.npz
Saved G:/signData\nptxt_val\17\0012.npz
Saved G:/signData\nptxt_val\17\0013.npz
Saved G:/signData\nptxt_val\17\0014.npz
Saved G:/signData\nptxt_val\17\0015.npz
Saved G:/signData\nptxt_val\17\0016.npz
Saved G:/signData\nptxt_val\17\0017.npz
Saved G:/signData\nptxt_val\17\0018.npz
Saved G:/signData\nptxt_val\17\0019.npz
Saved G:/signData\nptxt_val\17\0020.npz
Saved G:/signData\nptxt_val\17\0021.npz
Saved G:/signData\nptxt_val\17\0022.npz
Saved G:/signData\nptxt_val\17\0023.npz
Saved G:/signData\nptxt_val\17\0024.npz
Saved G:/signData\nptxt_val\17\0025.npz


In [27]:
 def getoutputdir(type="train"):
    return val_output_dir if type=='val' else output_dir

with open('wordtonum.json', 'r', encoding="UTF8") as json_file:
    words_dicts = json.load(json_file)

def load_data(file_name,type="train"):
    path=f"{getoutputdir(type)}/{file_name}"
    weight_path = f"{weight_dir}/{file_name}"
    data = np.load(path)
    weight_data = np.load(weight_path)
    #좌표값 로드
    wordCoordL = data['wordCoordL']
    wordCoordR = data['wordCoordR']
    wordCoordP = data['wordCoordP']
    #단어 뜻 호출
    ans = data['label'][0]
    # ans = ans.replace('\n', '')
    #해당 단어의 value 호출
    label = words_dicts[ans]
    weight = weight_data['weight']
    return wordCoordL, wordCoordR, wordCoordP, label ,weight



def load_word(person, start, num):
    coordLs = []
    coordRs = []
    coordPs = []
    labels = []
    checks = []
    for WNum in range(start, start + num):
        wordCoordL, wordCoordR, wordCoordP, label = load_data(person, WNum)
        coordLs.append(wordCoordL)
        coordRs.append(wordCoordR)
        coordPs.append(wordCoordP)
        labels.append(int(label[0]))
        checks.append((label[1:3]))

    return coordLs, coordRs, coordPs, labels, checks

In [34]:
#make lable



person_blacklist=[]
word_dict={}
num=1
for person in os.listdir(train_morpheme_dir):
    for word_morpheme in os.listdir(os.path.join(train_morpheme_dir, person)):
        if "F" in word_morpheme:
            file_path = os.path.join(train_morpheme_dir, person, word_morpheme)
            morpheme_file_path = os.path.join(train_morpheme_dir, person, word_morpheme)
            with open(morpheme_file_path, 'r', encoding="UTF8") as morpheme_file:
                data = json.load(morpheme_file)
                try:
                    name = data['data'][0]['attributes'][0]['name']
                    name = name.replace("\n", "")
                    if name[-1].isdigit():
                        name = name[:-1]
                    if name in word_dict:
                        continue
                    else:
                        print(name)
                        word_dict[name]=num
                        num+=1
                except IndexError as e:
                    name=False
                    continue
               
    with open('wordtonum.json', 'w') as json_file:
        json.dump(word_dict, json_file)

고민
뻔뻔
수어
남아
눈
독신
음료수
발가락
슬프다
자극
안타깝다
어색하다
여아
외국인
영아
신사
뉴질랜드
나사렛대학교
알아서
장애인
열아홉번째
침착
성실
학교연혁
싫어하다
급하다
필기시험
병문안
검사
결승전
낚시터
낚시대
당뇨병
독서
매표소
면역
감기
배드민턴
변비
병명
보건소
불면증
불행
붕대
사위
설사
성병
방충
소화제
손녀
손자
수면제
수집가
여행지
예식장
올림픽경기
회복
첫번째
운동경기
입원
재혼
진단서
축구장
치료
치료법
친아들
퇴원
한약
한약방
빈혈
화상
가래떡
고깃국
고추
고추가루
사골
배추국
꽈베기
벌꿀
꿀물
냄비
찬물
다과
지방경찰청장
된장찌게
돼지고기
두부
딸기
떡국
라면
막걸리
무
밥그릇
밥솥
보신탕
부엌
소불고기
비빔밥
사과
사이다
달다
소주
술잔
식당
쌀가루
냉커피
양식
위스키
술
용품
유리그릇
짬뽕
참외
칼국수
커피
콜라
콩가루
통조림
와인
백설기
가죽신
군복
포켓
양산
운동화
잠옷
이불
에어컨
행거
월세
전세
치약
가수
체감
경고
경찰서
공식
과장
관광버스
교수
교장
찻길
독일어
반환
음주운전
동아리
디자이너
마을버스
구급차
개입하지않는다
사무
신입학
여객선
여행사
기차역
예술가
외국어
요리사
강요
운동선수
퇴임
의사
이력서
자가용
재택근무
연령제한
주차장
채팅
출근
코치
터미널
근무
통역사
퇴사
사직
판결
평가
두번째
항공사
교통안전교육
화상채팅
회식
교통기본교육
후원
희생
간호사
교통법교육
계약금
고용
사업장
버스값
구직
논
대기업
사육장
백화점
압류
거지
수입
신용카드
백수
아르바이트
샛길
일시불
주유소
집주인
한국은행
경찰청장
가정교육
고등학교
고등학생
교무실
개학
교실
교육자
국어학
낙제
예절
독서실
등교
맹학교
복습
분필
상장
서점
학업
실습
여학교
여학생
연구소
연구실
예습
유학
의학
지도
재학
연구
학부모
학식
훈련
경기도
경북
남산
울릉도
울산
전남
청주
충북
충남
불경
성경책
신학
장로교
일요일
축하
회개
소감
장난
장인
희생자
디자인
돌무덤
쇠기둥
동화책
목공예
문학
문화유산
음악가
예술
초상화
토기
흑백영화
공

In [35]:
with open('wordtonum.json', 'r', encoding="UTF8") as json_file:
    data = json.load(json_file)
    print(data)
    

{'고민': 1, '뻔뻔': 2, '수어': 3, '남아': 4, '눈': 5, '독신': 6, '음료수': 7, '발가락': 8, '슬프다': 9, '자극': 10, '안타깝다': 11, '어색하다': 12, '여아': 13, '외국인': 14, '영아': 15, '신사': 16, '뉴질랜드': 17, '나사렛대학교': 18, '알아서': 19, '장애인': 20, '열아홉번째': 21, '침착': 22, '성실': 23, '학교연혁': 24, '싫어하다': 25, '급하다': 26, '필기시험': 27, '병문안': 28, '검사': 29, '결승전': 30, '낚시터': 31, '낚시대': 32, '당뇨병': 33, '독서': 34, '매표소': 35, '면역': 36, '감기': 37, '배드민턴': 38, '변비': 39, '병명': 40, '보건소': 41, '불면증': 42, '불행': 43, '붕대': 44, '사위': 45, '설사': 46, '성병': 47, '방충': 48, '소화제': 49, '손녀': 50, '손자': 51, '수면제': 52, '수집가': 53, '여행지': 54, '예식장': 55, '올림픽경기': 56, '회복': 57, '첫번째': 58, '운동경기': 59, '입원': 60, '재혼': 61, '진단서': 62, '축구장': 63, '치료': 64, '치료법': 65, '친아들': 66, '퇴원': 67, '한약': 68, '한약방': 69, '빈혈': 70, '화상': 71, '가래떡': 72, '고깃국': 73, '고추': 74, '고추가루': 75, '사골': 76, '배추국': 77, '꽈베기': 78, '벌꿀': 79, '꿀물': 80, '냄비': 81, '찬물': 82, '다과': 83, '지방경찰청장': 84, '된장찌게': 85, '돼지고기': 86, '두부': 87, '딸기': 88, '떡국': 89, '라면': 90, '막걸리': 91, '무': 92, '밥그릇': 93, '밥솥': 94, '보

In [36]:
with open('wordtonum.json', 'r', encoding="UTF8") as json_file:
    data = json.load(json_file)
    print(data['신부'])
    

2022
